In [122]:
import gym
import ptan
import argparse
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [123]:
#set parameters
num_gps = 3
num_slots = 4
num_pre_booked = 2
num_to_book = 3

In [124]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01
BATCH_SIZE = 128
NUM_ENVS = 50

REWARD_STEPS = 4
CLIP_GRAD = 0.1

In [125]:
class PGN(nn.Module):
    def __init__(self, input_size, n_actions):
        super(PGN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [126]:
class SchedulerEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self):
        
        #set parameters for the day
        self.num_gps = num_gps
        self.num_slots = num_slots
        self.num_pre_booked = num_pre_booked
        self.num_to_book = num_to_book

        #set action space this format of the diary
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(num_slots, num_gps), dtype=np.int32)
        
        #set observation space 
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(num_slots, num_gps), dtype=np.int32)
   
    #creates daily diary for each gp and randomly populates prebooked appointments
    def reset(self):

        #creates zero filled dataframe with row per time slot and column per gp
        self.state = np.zeros((self.num_slots, self.num_gps),dtype=float)

        #randomly enters a 1 for each pre booked appointments
        while self.num_pre_booked>=0:
            self.num_pre_booked -= 1
            row_to_update = np.random.randint(self.num_slots, size=1)
            col_to_update = np.random.randint(self.num_gps, size=1)
            #self.state.at[row_to_update[0],col_to_update[0]]=1
            self.state[row_to_update,col_to_update]=1

        #resets parameters for new episode
        self.done = False
        self.reward = 0
        self.num_to_book = num_to_book
        self.num_pre_booked = num_pre_booked

        return self.state

    def step(self, action):
    
        tot_appts = self.num_pre_booked + self.num_to_book
        print('total appts = ', tot_appts)
        new_state = self.state[action]
        print('appts in diary = ', new_state.sum())
        print(new_state)
        
        if tot_appts == new_state.sum():
            self.reward +=1

        self.done = True
        info = {}

        return new_state, self.reward, self.done, info
        

In [127]:
env = SchedulerEnv()
state = env.reset()
state.size

12

In [128]:
device = torch.device("cpu")
net = PGN(env.state.size, 5)
print(net)

PGN(
  (net): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=5, bias=True)
  )
)


In [129]:
#convert numpy array to tensor for input
def tensor_convert(x):
    return torch.from_numpy(x).float()

In [130]:
agent = ptan.agent.PolicyAgent(lambda x: net(x)[0], apply_softmax=True, device=device)

nn_input = torch.flatten(tensor_convert(state))
print(nn_input)

action = net(nn_input)

#optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)

tensor([0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.])


In [131]:
action

tensor([ 0.1430, -0.0827,  0.0813, -0.0842,  0.0230], grad_fn=<AddBackward0>)

In [136]:
env.action_space

Box([[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]], [[1 1 1]
 [1 1 1]
 [1 1 1]
 [1 1 1]], (4, 3), int32)

In [132]:
states_v, actions_t, vals_ref_v = unpack_batch(batch, net, device=device)


    

NameError: name 'unpack_batch' is not defined

In [ ]:
optimizer.zero_grad()
    logits_v, value_v = net(states_v)
    loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v)

    log_prob_v = F.log_softmax(logits_v, dim=1)
    adv_v = vals_ref_v - value_v.detach()
    log_prob_actions_v = adv_v * log_prob_v[range(BATCH_SIZE), actions_t]
    loss_policy_v = -log_prob_actions_v.mean()

    prob_v = F.softmax(logits_v, dim=1)
    entropy_loss_v = ENTROPY_BETA * (prob_v * log_prob_v).sum(dim=1).mean()

    # calculate policy gradients only
    loss_policy_v.backward(retain_graph=True)
    grads = np.concatenate([p.grad.data.cpu().numpy().flatten()
                            for p in net.parameters()
                            if p.grad is not None])

    # apply entropy and value gradients
    loss_v = entropy_loss_v + loss_value_v
    loss_v.backward()
    nn_utils.clip_grad_norm_(net.parameters(), CLIP_GRAD)
    optimizer.step()
    # get full loss
    loss_v += loss_policy_v

In [ ]:
def train(env, replay_memory, model, target_model, done):
    learning_rate = 0.7 # Learning rate
    discount_factor = 0.618

    MIN_REPLAY_SIZE = 1000
    if len(replay_memory) < MIN_REPLAY_SIZE:
        return

    batch_size = 64 * 2
    mini_batch = random.sample(replay_memory, batch_size)
    current_states = np.array([encode_observation(transition[0], env.observation_space.shape) for transition in mini_batch])
    current_qs_list = model.predict(current_states)
    new_current_states = np.array([encode_observation(transition[3], env.observation_space.shape) for transition in mini_batch])
    future_qs_list = target_model.predict(new_current_states)

    X = []
    Y = []
    for index, (observation, action, reward, new_observation, done) in enumerate(mini_batch):
        if not done:
            max_future_q = reward + discount_factor * np.max(future_qs_list[index])
        else:
            max_future_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = (1 - learning_rate) * current_qs[action] + learning_rate * max_future_q

        X.append(encode_observation(observation, env.observation_space.shape))
        Y.append(current_qs)
    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)
    
def encode_observation(observation, n_dims):
    return observation

In [ ]:
    epsilon = 1 # Epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
    max_epsilon = 1 # You can't explore more than 100% of the time
    min_epsilon = 0.01 # At a minimum, we'll always explore 1% of the time
    decay = 0.01

    # 1. Initialize the Target and Main models
    # Main Model (updated every 4 steps)
    model = agent(env.observation_space.shape, env.action_space.n)
    # Target Model (updated every 100 steps)
    target_model = agent(env.observation_space.shape, env.action_space.n)
    target_model.set_weights(model.get_weights())

    replay_memory = deque(maxlen=50_000)

    target_update_counter = 0

    # X = states, y = actions
    X = []
    y = []

    steps_to_update_target_model = 0

    for episode in range(train_episodes):
        total_training_rewards = 0
        observation = env.reset()
        print('starting', observation)
        free_gps = pd.DataFrame(observation).isin([0]).all().sum()
        print("Number of free GPs: ", free_gps)
        done = False
        while not done:
            steps_to_update_target_model += 1
            #if True:
            #    env.render()

            random_number = np.random.rand()
            # 2. Explore using the Epsilon Greedy Exploration Strategy
            if random_number <= epsilon:
                # Explore
                action = env.action_space.sample()
            else:
                # Exploit best known action
                # model dims are (batch, env.observation_space.n)
                encoded = encode_observation(observation, env.observation_space.shape[0])
                encoded_reshaped = encoded.reshape([1, encoded.shape[0], encoded.shape[1]])
                predicted = model.predict(encoded_reshaped).flatten()
                action = np.argmax(predicted)
            new_observation, reward, done, info = env.step(action)
            replay_memory.append([observation, action, reward, new_observation, done])

            # 3. Update the Main Network using the Bellman Equation
            if steps_to_update_target_model % 4 == 0 or done:
                train(env, replay_memory, model, target_model, done)

            observation = new_observation
            total_training_rewards += reward

            if done:
                print('Total training rewards: {} after n steps = {} with final reward = {}'.format(total_training_rewards, episode, reward))
                free_gps = pd.DataFrame(observation).isin([0]).all().sum()
                print("Number of free GPs: ", free_gps)
                print('end', episode, observation)
                total_training_rewards += 1

                if steps_to_update_target_model >= 100:
                    print('Copying main network weights to the target network weights')
                    target_model.set_weights(model.get_weights())
                    steps_to_update_target_model = 0
                break

        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay * episode)
    env.close()

In [133]:
test1 = env.action_space.sample()
test1

array([[1, 1, 1],
       [1, 1, 0],
       [0, 0, 0],
       [1, 0, 0]], dtype=int32)

In [ ]:
action.sum()